# SARAH Toolbox
## Collection of models to be run from a docker image
Launch virtual docker image: 
[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/SimplifiedDMSSSFDM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)
## Implemented models
Each model is to be run in a specific virtual machine. Follow the binder button in each github repo
* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/SM/HighScale` (For RGE running: ./butler SM/HighScale) [Repo](https://github.com/restrepo/SM-Toolbox/)
* `SARAH/Models/SimplifiedDM/DFDM` (This image)
* `SARAH/Models/SimplifiedDM/SDFDM` (This image)

## Instructions to compile the model
In `SPHENO` and `micromegas`

See possible analysis based on the models in  [./tests](./tests) folder.

Below we define the model to be compiled:

In [24]:
MODEL=SimplifiedDMSSSFDM

To better control in the outputs, it is recomended that the commands be executed [from a terminal](../terminals/1). Only possible errors are to be shown below

In [2]:
cd SPHENO
make Model=$MODEL > /dev/null
cd ..

make a work dir

In [30]:
mkdir -p test_compilation
cd test_compilation

In [31]:
LHAINPUT=../SPHENO/$MODEL/Input_Files/LesHouches.in.$MODEL
if [ -f ../SARAH/Models/$MODEL/LesHouches.in.$MODEL ]; then
   LHAINPUT=../SARAH/Models/$MODEL/LesHouches.in.$MODEL
fi
cp $LHAINPUT . #only a number of limited charactes accepted as input

Run SPheno and generate LHA output file. However, there is one caveat in `MadGraph`: `MadEvent` has problems with reading the HiggsBounds specific blocks in the SPheno spectrum file (`HiggsBoundsInputHiggsCouplingsFermions` and `HiggsBoundsInputHiggsCouplingsBosons`). If these blocks are included, MadEvent won’t accept the file. Therefore, The `HiggsBounds` blocks need to be disabled:

In [32]:
sed -ri 's/(^520) 1/\1 0/' LesHouches.in.$MODEL

In fact:

In [33]:
grep ^520 LesHouches.in.$MODEL

520 0.              # Write effective Higgs couplings (HiggsBounds blocks): put 0 to use file with MadGraph! 


We now run the spectrum generator

In [34]:
../SPHENO/bin/SPheno$MODEL LesHouches.in.$MODEL

 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


with the LHA output:

In [35]:
ls SPheno.spc.$MODEL

SPheno.spc.SimplifiedDMSSSFDM


## micrOMEGAS

In [36]:
cd ../micromegas
make > /dev/null
make > /dev/null
cd $MODEL
make main=CalcOmega.cpp > /dev/null

###  Include Indirect Detection

In [42]:
micromegas_with_DDetection=CalcOmega_with_DDetection_MOv4.2.cpp 
IDmsg="======== Indirect Detection ========"
if [ -f "$micromegas_with_DDetection" ]  && [ ! "$(grep "$IDmsg" "$micromegas_with_DDetection" )" ]; then
    tmpfile=$(mktemp /tmp/CalcOmega_UP.XXXXXX)
    grep -B 1000 'fclose(channels);' "$micromegas_with_DDetection" | grep -v 'fclose(channels);' > "$tmpfile"
    echo "
{
  double Emin=1,/* Energy cut  in GeV   */  sigmaV;
  double vcs_gz,vcs_gg;
  char txt[100];
  double SpA[NZ],SpE[NZ],SpP[NZ];
  double FluxA[NZ],FluxE[NZ],FluxP[NZ];
  double * SpNe=NULL,*SpNm=NULL,*SpNl=NULL;
  double Etest=Mcdm/2;

  printf(\"\n$IDmsg\n\"); 
  sigmaV=calcSpectrum(1+2+4,SpA,SpE,SpP,SpNe,SpNm,SpNl ,&err);

}" >> "$tmpfile"

  grep -A 1000 'fclose(channels);' "$micromegas_with_DDetection" >> "$tmpfile"
  mv "$tmpfile" "$micromegas_with_DDetection"
else 
  echo "$IDmsg"
  echo "already included"
fi

======== Indirect Detection ========
already included


Compile `micrOMEGAS` with direct detection

In [43]:
make main="$micromegas_with_DDetection" > /dev/null

In [39]:
cd ../../test_compilation
../micromegas/$MODEL/$(basename "$micromegas_with_DDetection" .cpp) SPheno.spc.$MODEL


Masses of odd sector Particles:
~Ss  : MSs   =    28.4 || ~Re  : MRe   =   400.0 || 
PROCESS: ~Ss,~Ss->AllEven,1*x{,h,g,A,Z,Wp,Wm,nu1,Nu1,nu2,Nu2,nu3,Nu3,d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3,e1,E1,e2,E2,e3,E3
PROCESS: Z->2*x
PROCESS: Wp->2*x
PROCESS: ~Ss,~Ss->Wm,E1,nu1
Delete diagrams with %Z+W<1
Xf=1.61e+01 Omega h^2=2.23e+01

# Channels which contribute to 1/(omega) more than 1%.
# Relative contributions in % are displayed
   85% ~Ss ~Ss ->d3 D3 
    8% ~Ss ~Ss ->e3 E3 
    4% ~Ss ~Ss ->u2 U2 
    2% ~Ss ~Ss ->g g 

==== Calculation of CDM-nucleons amplitudes  =====
         TREE LEVEL
PROCESS: QUARKS,~Ss->QUARKS,~Ss{d1,D1,d2,D2,d3,D3,u1,U1,u2,U2,u3,U3
Delete diagrams with _S0_!=1,_V5_,A
CDM-nucleon micrOMEGAs amplitudes:
proton:  SI  2.479E-09  SD  0.000E+00
neutron: SI  2.510E-09  SD  0.000E+00
         BOX DIAGRAMS
CDM-nucleon micrOMEGAs amplitudes:
proton:  SI  2.479E-09  SD  0.000E+00
neutron: SI  2.510E-09  SD  0.000E+00
CDM-nucleon cross sections[pb]:
 proton  SI 2.517E-09  SD

##  Madgraph

We are using [MG5_aMC_v2_3_3](https://launchpad.net/mg5amcnlo/2.0/2.3.x/+download/MG5_aMC_v2.3.3.tar.gz).

We follow here clossely the SARAH Manual: _Exploring new models in all detail with_ `SARAH`, [arxiv.org:1503.04200](https://arxiv.org/pdf/1503.04200.pdf), Sec. "Dilepton analysis with MadGraph", pag. 118, but adapted to our problem.

The command to generate the UFO files is

 `MakeUFO []`
 
Within the SARAH toolbox used here, this `Mathematica` command is automatically executed and the resulting code is copied into the proper `MadGraph` folder at `madgraph/models/$MODEL`

Here, we use an input file for MadGraph: `install_and_check.mdg`  which contains the following lines.
(If all tests are passed hit `<ENTER>`)

In [16]:
cat << EOF > install_and_check.mdg #Creates the file with the folloing content before EOF
#install pythia-pgs  
## Check your root installation and install Delphes
#install Delphes
## Check SARAH model
import model $MODEL -modelname
# p is already defined if imported from	MadGraph console
define p g d1 d1bar d2 d2bar u1 u1bar u2 u2bar
check p p > e2 e2bar
EOF

In [17]:
cd ../madgraph  
./bin/mg5_aMC ../test_compilation/install_and_check.mdg 2>/dev/null | tail -n20
cd ../test_compilation

************************************************************
load MG5 configuration from input/mg5_configuration.txt 
fastjet-config does not seem to correspond to a valid fastjet-config executable (v3+). We will use fjcore instead.
 Please set the 'fastjet'variable to the full (absolute) /PATH/TO/fastjet-config (including fastjet-config).
 MG5_aMC> set fastjet /PATH/TO/fastjet-config

set lhapdf to lhapdf-config
Using default eps viewer "evince". Set another one in ./input/mg5_configuration.txt
Using default web browser "firefox". Set another one in ./input/mg5_configuration.txt
import /home/restrepo/prog/toolbox/toolbox/test_compilation/install_and_check.mdg
The import format was not given, so we guess it as command
import model SimplifiedDMSDFDM -modelname
Defined multiparticle all = g a nu1 nu2 nu3 nu1bar nu2bar nu3bar d1 d2 d3 u1 u2 u3 d1bar d2bar d3bar u1bar u2bar u3bar h z wp wpc re e1 e2 e3 chi1 chi2 chi3 rebar e1bar e2bar e3bar
define p g d1 d1bar d2 d2bar u1 u1bar u2 u2bar
De

Now we can generate some processes. We just check that the lightest righ handed neutrino does no couple with the photon.
The possible VBF processes is in a commented line.

In [18]:
cat << EOF > VBF.mdg #Creates the file with the folloing content before EOF 
#See: http://www.niu.edu/spmartin/madgraph/madsyntax.html
import model $MODEL -modelname
#p is already defined if imported from	MadGraph console
define p g d1 d1bar d2 d2bar u1 u1bar u2 u2bar
define j g d1 d1bar d2 d2bar u1 u1bar u2 u2bar   
#Just check that right-handed neutrino does no couples to foton:
generate p p > e2 e2bar QCD=0
output MadOut -f
exit
EOF
../madgraph/bin/mg5_aMC VBF.mdg  2> /dev/null | tail -n15

save configuration file to /home/restrepo/prog/toolbox/toolbox/test_compilation/MadOut/Cards/me5_configuration.txt
INFO: Use Fortran compiler gfortran 
INFO: Generate jpeg diagrams 
INFO: Generate web pages 
Output to directory /home/restrepo/prog/toolbox/toolbox/test_compilation/MadOut done.
Type "launch" to generate events from this process, or see
/home/restrepo/prog/toolbox/toolbox/test_compilation/MadOut/README
Run "open index.html" to see more information about this process.
exit
 
quit
 

 



In the first of line `VBF.mdg`, we import the model in `MadGraph`. The option `-modelname` is used to keep the names of the particles as given in the model files. By default, `MadGraph` will try to use the default naming conventions. However, this would fail for this model. We define a multi-particle called `p` which consists of gluon and all the light quarks. If we import the model direcly from the `MadGraph` console, `p` is definded automatically from the new model name particles.

The output for MadEvent is written to a new subdirectory `test_compilation/MadOut`  and we close `MadGraph` when it is done via exit. 

After MadGraph has created the output for MadEvent and finished, we can enter the new subdirectory `test_compilation/MadOut`. The important settings to generate events are done via the files in the Cards-directory: the file `param_card.dat` is used to give the input for all parameters, `run_card.dat` controls the event generations. In the last file, the user can for instance set the beam type and energy, define the renormalization scale, apply cuts, and fix the number of events. We want to use, of course, the spectrum file as written by SPheno. 

When we have the spectrum file in the correct form, we can copy this file to the Cards directory as
`param_card.dat`.

In [19]:
cp SPheno.spc.$MODEL MadOut/Cards/param_card.dat 

Changes in any of the input parameters, require to update the `SPheno` output and overwrite the `MadOut/Cards/param_card.dat` file

In [26]:
cat << EOF > launch.mdg
launch MadOut -f
EOF
../madgraph/bin/mg5_aMC launch.mdg 2>/dev/null | grep ' Cross-section :'

     Cross-section :   825.3 +- 1.873 pb


## Go now to: 
* [main ipyrhon file](./tests/SimplifiedDM-SSSFDM.ipynb) → Singlet Scalar Singlet (charged) Fermion Dark Matter